In [55]:
# Install Dependencies
#!pip install scenedetect[opencv,progress_bar]

In [26]:
import pandas as pd
import numpy as np
import io
import os

from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from apiclient import errors

yt_url_prefix = 'https://www.youtube.com/watch?v='
output_folder = 'data/'

In [63]:
# GOOGLE DRIVE API CONSTANTS
flow = InstalledAppFlow.from_client_secrets_file('../credentials.json', ['https://www.googleapis.com/auth/drive'])
creds = flow.run_local_server(port=0)
folder_id = '1UnDIe4VHMM8bZzIfKEG8NJdQGnNrrxEG'
drive_api = build('drive', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=786879558582-eh0bho168rp2k1qord3av8r5u10q64sl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62857%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=WCQJhBkxkDH8MbHyamBYzoV3M3wqJN&access_type=offline


In [109]:
# Helpful function
def get_file_list_from_folder(service, folder_id):
    """Print files belonging to a folder.

    Args:
    service: Drive API service instance.
    folder_id: ID of the folder to print files from.
    """
    kwargs = {
        "q": "'{}' in parents".format(folder_id)
    }
    request = service.files().list(**kwargs)
    files = []
    while request is not None:
        response = request.execute()
        # Do stuff with response['files']
        files.extend(response['files'])
        request = service.files().list_next(request, response)
    return files

def download_file(service, file_id, out_file_dir):
    with open(out_file_dir, 'wb') as fo:
        a = service.files().get_media(fileId=file_id).execute()
        fo.write(a)


In [23]:
uploaded = get_file_list_from_folder(drive_api, folder_id)

In [107]:
#uploaded

In [110]:
download_file(drive_api, uploaded[0]['id'], uploaded[0]['name'])